In [39]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("/Users/peggy/Documents/109-2(2-2)/machine_learning/MLGame/games/snake/log/snake.pickle", "rb")
data = pickle.load(file)
file.close()
type(data['ml'])

dict

In [42]:
game_info = data["ml"]["scene_info"]
game_command = data["ml"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'snake_head': (40, 40), 'snake_body': [(40, 30), (40, 20), (40, 10)], 'food': (40, 160)}, {'frame': 1, 'status': 'GAME_ALIVE', 'snake_head': (40, 50), 'snake_body': [(40, 40), (40, 30), (40, 20)], 'food': (40, 160)}, {'frame': 2, 'status': 'GAME_ALIVE', 'snake_head': (30, 50), 'snake_body': [(40, 50), (40, 40), (40, 30)], 'food': (40, 160)}, {'frame': 3, 'status': 'GAME_ALIVE', 'snake_head': (20, 50), 'snake_body': [(30, 50), (40, 50), (40, 40)], 'food': (40, 160)}, {'frame': 4, 'status': 'GAME_ALIVE', 'snake_head': (10, 50), 'snake_body': [(20, 50), (30, 50), (40, 50)], 'food': (40, 160)}, {'frame': 5, 'status': 'GAME_ALIVE', 'snake_head': (10, 60), 'snake_body': [(10, 50), (20, 50), (30, 50)], 'food': (40, 160)}, {'frame': 6, 'status': 'GAME_ALIVE', 'snake_head': (20, 60), 'snake_body': [(10, 60), (10, 50), (20, 50)], 'food': (40, 160)}, {'frame': 7, 'status': 'GAME_ALIVE', 'snake_head': (30, 60), 'snake_body': [(20, 60), (10, 60), (10, 50)], 'fo

In [43]:
g = game_info[1]

feature = np.array([g["snake_head"][0], g["snake_head"][1]])
print(feature)
print(game_command[1])
if game_command[1] == "UP": game_command[1] = 0
elif game_command[1] == "DOWN": game_command[1] = 1
elif game_command[1] == "LEFT": game_command[1] = 2
elif game_command[1] == "RIGHT": game_command[1] = 3
else: game_command[1] = 4 

[40 50]
2


In [44]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    
    feature = np.vstack((feature, [g["snake_head"][0], g["snake_head"][1]]))
    if game_command[i] == "UP": game_command[i] = 0
    elif game_command[i] == "DOWN": game_command[i] = 1
    elif game_command[i] == "LEFT": game_command[i] = 2
    elif game_command[i] == "RIGHT": game_command[i] = 3
    else: game_command[i] = 4 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 40  50]
 [ 30  50]
 [ 20  50]
 ...
 [270 200]
 [280 200]
 [290 200]]
(1339, 2)
[4 2 2 ... 3 3 3]
(1339,)


# MLPClassifier

In [45]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7)
MLPC = MLPClassifier(random_state=1, max_iter=300).fit(x_train, y_train)

MLPC.predict_proba(x_test)
MLPC.predict(x_test)
MLPC.score(x_test, y_test)

file = open("model_MLPC.pickle", "wb")
pickle.dump(MLPC, file)
file.close()

In [46]:
print("Accuracy score (validation): {0:.3f}".format(MLPC.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, MLPC.predict(x_test)))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, MLPC.predict(x_test)))

Accuracy score (validation): 0.473
Confusion Matrix for Raandom Forests:
[[  7   0   0   0]
 [  0   0   0   9]
 [  0   0  66 128]
 [  0   0  75 117]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.00      0.00      0.00         9
           2       0.47      0.34      0.39       194
           3       0.46      0.61      0.52       192

    accuracy                           0.47       402
   macro avg       0.48      0.49      0.48       402
weighted avg       0.46      0.47      0.46       402



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
